In [1]:
from openai import OpenAI
import speech_recognition as sr
import os
import time
from gtts import gTTS
import playsound


client = OpenAI(api_key="sorry, this is private for a github repo.")

assistant = client.beta.assistants.create(
    name = "Lopçük",
    instructions = "Lopçük is a friendly assistant that helps you with your daily conversational tasks.",
    tools=[{"type": "code_interpreter"}],
    model = "gpt-3.5-turbo",
)

thread = client.beta.threads.create()


In [2]:
def speak(text):
    tts = gTTS(text=text, lang='en')
    file_path = os.path.abspath("data/audio.mp3")
    tts.save(file_path)
    playsound.playsound(file_path)


def speech_to_text():
    recognizer = sr.Recognizer()
    microphone = sr.Microphone()
    with microphone as source:
        print("Say something!")
        recognizer.adjust_for_ambient_noise(source, duration=1)
        audio = recognizer.listen(source)
    try:
        text = recognizer.recognize_google(audio, language="en-US")
        print("You said:", text)
        return text.lower()  
    except sr.UnknownValueError:
        speak("Sorry, I could not understand what you said.")
        return None
    except sr.RequestError as e:
        speak(f"Could not request results from Google Speech Recognition service; {e}")
        return None

def send_input_to_assistant(client, thread, input_text):
    message = client.beta.threads.messages.create(thread_id=thread.id, role="user", content=input_text)
    return message

def wait_for_response_completion(client, thread, run):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
        time.sleep(0.5)
    return run

def get_last_message(client, thread):
    response = client.beta.threads.messages.list(thread_id=thread.id, order="asc")
    if response.data:  
        last_message = response.data[-1].content[0].text.value
        return last_message
    else:
        last_message = "No messages found in the conversation thread."
        return last_message


def speak_last_message(client, thread):
    speak("Hello there, how can I help you today?")
    speak_last_message_loop(client, thread)

def speak_last_message_loop(client, thread, last_message=None):
    while True:
        
        user_input = speech_to_text()
        if user_input == "exit":
            speak("OK, Goodbye!")
            break
        if user_input is not None:
            send_input_to_assistant(client, thread, user_input)
            run = client.beta.threads.runs.create(thread_id=thread.id, assistant_id=assistant.id)
            run = wait_for_response_completion(client, thread, run)
            last_message = get_last_message(client, thread)
            speak(last_message)

speak_last_message(client, thread)


Say something!
You said: can you give me a brief information about Metallica
Say something!
You said: exit
